# Intro to AWS
### Lunch and Learn
---
#### Zach Smialek
#### 5.24.2018

# Overview
___
- What is AWS and how it works
- Getting Started
- Configuration
- Overview of services
- Building an IoT Application


# What is AWS
---
- Amazons cloud computing platform. 
- Bucket full of lego pieces.
- Pay for what you use
- Anything a developer would never need 

# Quick Tour of the Console
---
- Freewave Login: https://freewave.signin.aws.amazon.com/console
- **User**: See board
- **Password**: See board


# AWS Credentials
---
In order to use services via [aws cli](https://aws.amazon.com/cli/) or [aws sdk](https://aws.amazon.com/tools/) we need to put keys in ~/.aws/credentials. We will setup credentials towards the end.


In [14]:
import os

def writer(file, contents):
    """Helper function to write files.
    
    Args:
        file (string): File name to save contents to
        contents (string): File contents to be written.
    
    Returns:
        None
    """
    with open(file, 'a') as f:
        f.write(contents)

In [4]:
def credentials(key, secret):
    """Contructs the string to be written to file.
    
    Args:
        key (str): Is the aws_access_key_id generated when a user is 
            created through AWS IAM.
        secret (str): Is the aws_secret_access_key generated when a 
            user is created through AWS IAM.
        
    Returns:
        contents (str): Formatted string to be written to the
            credentials file.
            
            [default]
            aws_access_key_id = XXXXX
            aws_secret_access_key = XXXXX
    """
    aws_access_key_id = 'aws_acess_key_id = %s' % access_key
    aws_secret_access_key = 'aws_secret_access_key = %s' % secret
    
    contents = "[default]\n%s\n%s" % (aws_access_key_id, 
                                     aws_secret_access_key)
    return contents

def save_credentials(credentials):
    """Saves AWS credentials in the correct place
    
    Saves a file named credentials ~/.aws this file tells AWS
    who you can and which services you have rights to.
    
    Args:
        credentials (str): contents of file to be writtem.
        
    Returns:
        None
    """
    path = os.path.expanduser('~/.aws') + '/credentials'

    writer(path, credentials)

access_key = 'CHANGE_ME'
secret = 'CHANGE_ME'

save_credentials(credentials(access_key, secret))

# Services
---
Today the discussion well be centered a small subset of the service
- EC2
- S3
- IoT Core
- DynamoDB
- Cloudformation
- IAM

# EC2 - Elastic Compute Cloud
###### clever right?
---
EC2 is all about giving you a way to host virtual machines. One has an array of different OS and power
- Instance Type are VM's with different CPU, RAM, and networking capabilities
- VM's are sorted into five different categories
  - General
  - Compute optimized
  - Memory optimized
  - Accelerated Computing
  - Storage optimized

# S3 - Simple Storage Service
##### even more clever
---
As the name implies this service is centered around storage! 
- Buckets are where files get uploaded to. Easy peasey you put objects into buckets
- Buckets are global namespace names need to be unique.

In [4]:
import json

s3 = boto3.client('s3')

# TODO: Change bucket
BUCKET_NAME = 'fw-iot-face-detection-app'

BUCKET_POLICY = {
    'Version': '2012-10-17',
    'Statement': [{
        'Sid': 'GetImagesBucketPolicy',
        'Effect': 'Allow',
        'Principal': '*',
        'Action': ['s3:GetObject', 's3:PutObject'],
        'Resource': "arn:aws:s3:::%s/*" % BUCKET_NAME
    }]
}


def bucket_setup(bucket_name, bucket_policy):
    
    s3.create_bucket( Bucket=BUCKET_NAME,
                      CreateBucketConfiguration = {
                          'LocationConstraint': 'us-west-2'
                       },
                       ACL='private'
                     )

    bucket_policy = json.dumps(bucket_policy)

    s3.put_bucket_policy(Bucket=BUCKET_NAME, Policy=bucket_policy)


bucket_setup(BUCKET_NAME, BUCKET_POLICY)

# IoT Core 
---
IoT core gives us the ability to connected and interacted with devices. I rely heavily on this service to get data into AWS.
- MQTT
  - Wildcards are supported
- Broker
  - Publish
  - Subscribe
- Rules Engine
  - Ability to interact with other services
  - Responds to messages coming in to the Broker

# IoT Core
---
Lets say we have a topic some/sensor/location/reading we can define a rule.
<br>
- SELECT * FROM 'topic' 
- SELECT subset FROM 'topic
- SELECT *, topic() as topic, timestamp() as time FROM 'topic'

In [44]:
iot = boto3.client('iot')

def create_thing(thing_name):
    return iot.create_thing(
        thingName=thing_name,
        attributePayload={
            'attributes': {
                'event': 'lunchAndLearn'
            },
            'merge': True
        }
    )


In [43]:
def create_policy(policy_name, policy_doc):
    return iot.create_policy(
        policyName=policy_name,
        policyDocument=policy_doc
    )

In [53]:
def file_path(base_path, file_name):
    """Helper function to create path
    
    Args:
        base_path (str): current working directory
        file_name (str): name of file to be written
    
    Returns: 
        (str): Full path of a file to be written.
    
    """
    return base_path + file_name


def save_keys(path, key_pair):
    """Save private and public key to current directory
    
    Args:
        path (str): The path of the file with the file identifier
        key_pair (dict): Public and private key
        
        'keyPair': {
            'PublicKey': 'string',
            'PrivateKey': 'string'
        }
        
    Returns:
        None
    
    """
    private_key = (file_path(path, 'private.pem.key'), key_pair['PrivateKey'])
    public_key = (file_path(path, 'public.pem.key'), key_pair['PublicKey'])
    
    to_write = [private_key, public_key]
    
    for contents in to_write:
        writer(contents[0], contents[1])
    

    
def save_cert(path, cert):
    """Save cert to file system.
    
    Args:
        path (str): Path of the file to be written
        cert (str): Pem cert generated from AWS.
        
    Returns:
        None
    
    """
    
    file_name = file_path(path, 'certificate.pem.crt')
    writer(file_name, cert)


def create_keys_cert():
    """Creates keys and cert needed to connect things to the broker.
    
    Args:
        None
    
    Returns:
        cert_data (dict)
        
    {
        'certificateArn': 'string',
        'certificateId': 'string',
        'certificatePem': 'string',
        'keyPair': {
            'PublicKey': 'string',
            'PrivateKey': 'string'
        }
    }
    
    """
    cert_data = iot.create_keys_and_certificate(
        setAsActive=True
    )
    
    base_name = '/FWCam-Del-'
    path = '%s%s' % (os.getcwd(), base_name)
    
    save_cert(path, cert_data['certificatePem'])
    save_keys(path, cert_data['keyPair'])
    
    return cert_data

In [51]:
def principal_attachment(thing_name, policy_name, cert_arn):
    """Attaches thing and policy to the generated cert.
    
    The principal is what has rights or access to the service. In this case the device hand
        shakes with the broker before connecting. Then the "device" as access to the service.
        
    Args:
        thing_name (str): Name of the that that was recently created.
        policy_name (str): Name of the policy that was recently created.
        cert_arn (str): The certs amazon resource name.
        
    Returns:
        None
    """
    
    iot.attach_thing_principal(
        thingName=thing_name,
        principal=cert_arn
    )
    
    iot.attach_policy(
        policyName=policy_name,
        target=cert_arn
    )
    

In [52]:
def setup_thing(thing_name, policy_name):
    """Creates a IoT thing
    
    Args:
        thing_name (str): name of thing there is a constraint letters, numbers
            _, -
    
    Returns:
        None
    """
    POLICY_DOC = '''{
      "Version": "2012-10-17",
      "Statement": [
        {
          "Action": [
            "iot:Publish",
            "iot:Subscribe",
            "iot:Connect",
            "iot:Receive"
          ],
          "Effect": "Allow",
          "Resource": [
            "*"
          ]
        }
      ]
    } '''
    
    create_thing(thing_name)
    create_policy(policy_name, POLICY_DOC)
    cert_data = create_keys_cert()
    principal_attachment(thing_name, policy_name, cert_data['certificateArn'])
    
    
thing_name = 'FW-camera-2'
policy_name = 'LunchAndLearnDELME'
setup_thing(thing_name, policy_name)

# DynamoDB
---
NoSQL key value store database that is super flexible.
- Key Schema
  - Hash Key
  - Range Key (optional)
  - Global secondary index (optional)
  - Local secondaryh index (optional)
- Streams 
  - Transaction log
- Querying
  - Design schema around query needs
  - Hash key **always** needs to be in query statement
- Provisioned Throughput
  - Read capacity units
  - Write capacity units

In [25]:
client = boto3.client('dynamodb')

def create_table(table_name):
    """Creates a dynamodb table
    
    Simple example on creating a table in DynamoDB with Hash and Range key
    
    Args:
        table_name (str): name of the DynamoDB table
        
    Returns:
        response (dict): 
    """
    
    key_schema = {'hash': 'SensorName', 'range': 'Timestamp'}
    
    response = client.create_table(
        AttributeDefinitions=[
            {
                'AttributeName': key_schema['hash'],
                'AttributeType': 'S',
            },
            {
                'AttributeName': key_schema['range'],
                'AttributeType': 'N'
            }
        ],
        
        TableName= table_name,
        
        KeySchema=[
            {
                'AttributeName': key_schema['hash'],
                'KeyType': 'HASH'
            },
            {
                'AttributeName': key_schema['range'],
                'KeyType': 'RANGE'
            }
        ],
        
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )

create_table('FW.LunchAndLearn.SensorData')

In [ ]:
dynamodb_item = { 
    'SensorName': {
        'S': 'FakeThing'
    },
    'Timestamp': { 
        'N': '1234567894'
    },
    'SensorReading': {
        'N': '46'
    }
}

def insert_item(table_name, item):
    """Inserts an item in a given table
    
    The primary (hash) key needs to be in the item to insert
    
    Args:
        table_name (str):
        item (dict): item to insert into table
        
    Returns:
        response (dict):
    """
    return client.put_item(
        TableName=table_name,
        Item=item
    )

insert_item('FW.LunchAndLearn.SensorData', dynamodb_item)

In [ ]:
from boto3.dynamodb.conditions import Key, Attr

dynamodb = boto3.resource('dynamodb')

def query_table(hash_key, key_value):
    """Queries table for a given key
    
    Args:
        table_name (str): Name of the table
        hash_key (str): Primary key of the table
        range_key (str): Range key of the table
    
    Returns:
        response (dict):
        
    """
    table = dynamodb.Table('FW.LunchAndLearn.SensorData')
    
    return table.query(
        KeyConditionExpression=Key(hash_key).eq(key_value)
    )

print(query_table('SensorName', "FakeThing")['Items'])

# Cloudformation
---
A very very awesome service that allows you to create a stack via a JSON or YAML template. A stack is all the service you need for a given project. 
<br><br>
**ZumDash v2 stack**
- API Gateway
- Lambda
- DynamoDB
- IoT Core
- Cognito
- S3
- Cloudfront

# Cloudformation - DynamoDB Table
---
{
  "AWSTemplateFormatVersion" : "2010-09-09",
  "Resources" : {
    "myDynamoDBTable" : {
      "Type" : "AWS::DynamoDB::Table",
      "Properties" : {
        "AttributeDefinitions" : [
          {
            "AttributeName" : "Album",
            "AttributeType" : "S"   
          },
          {
            "AttributeName" : "Artist",
            "AttributeType" : "S"
          },
          {
            "AttributeName" : "Sales",
            "AttributeType" : "N"
          },
          {
            "AttributeName" : "NumberOfSongs",
            "AttributeType" : "N"
          }
        ],
        "KeySchema" : [
          {
            "AttributeName" : "Album",
            "KeyType" : "HASH"
          },
          {
            "AttributeName" : "Artist",
            "KeyType" : "RANGE"
          }
        ],
        "ProvisionedThroughput" : {
          "ReadCapacityUnits" : "5",
          "WriteCapacityUnits" : "5"
        },
        "TableName" : "myTableName"
      }
    }
  }
}